- sections : '정치', '경제', '사회'
- sections = ["pol", "eco","soc"]
- section_ids = ["100", "101","102"]

In [16]:
import requests
from bs4 import BeautifulSoup
import bs4.element
import datetime
import warnings
warnings.filterwarnings('ignore')
import re
import pandas as pd
import time
from datetime import datetime

In [17]:
news_titles = []
news_urls =[]
news_contents =[]
news_dates = []

In [18]:
# ConnectionError방지
headers = {'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36'}

# BeautifulSoup 객체 생성
def get_soup_obj(url):
    res = requests.get(url, headers = headers, verify=False)
    soup = BeautifulSoup(res.text,'lxml')
    
    return soup

In [19]:
# 뉴스의 기본 정보 가져오기
def get_top3_news_info():
    # 해당 분야 상위 뉴스 목록 주소
    day = datetime.today().strftime("%Y%m%d")
    base_url = "https://news.naver.com/main/list.naver?mode=LSD&mid=sec&sid1=001&listType=paper&date="+day
    
    # 해당 분야 상위 뉴스 HTML 가져오기
    soup = get_soup_obj(base_url)
  
    # 해당 분야 상위 뉴스 3개 가져오기
    # lis3 = soup.find('ul', class_='type06_headline').find_all('li', limit=3)
    lis3 = soup.find('ul', class_='type06_headline')

    for li in lis3:
        
        title = li.img.attrs.get('alt') if li.img else li.a.text.replace("\n", "").replace("\t","").replace("\r","")
        news_url = li.a.attrs.get('href')
        
        news_titles.append(title)
        news_urls.append(news_url)
        
    return news_urls

In [20]:
def get_news_contents(url):
    news_html = get_soup_obj(url)
    content = news_html.select("div#dic_area")
    if content == []:
        content = news_html.select("#articeBody")
        
    # 기사 텍스트만 가져오기
    # list합치기
    content = ''.join(str(content))

    # html태그제거 및 텍스트 다듬기
    pattern1 = '<[^>]*>'
    content = re.sub(pattern=pattern1, repl='', string=content)
    pattern2 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
    content = content.replace(pattern2, '')

    news_contents.append(content)

    html_date = news_html.select_one("div#ct > div.media_end_head > div.media_end_head_info > div span")
    news_date = html_date.attrs['data-date-time']
    news_dates.append(news_date)

    return news_contents

In [21]:
# 상위 3개 뉴스 크롤링
def get_naver_news_top3():
    # 뉴스의 기본 정보 가져오기
    get_top3_news_info()

    for i in news_urls:
        # 뉴스 본문 가져오기
        contents = get_news_contents(i)
        print(contents)

In [22]:
get_naver_news_top3()

None


TypeError: 'NoneType' object is not iterable

In [ ]:
news_df = pd.DataFrame({'date':news_dates,'title':news_titles,'link':news_urls,'content':news_contents})

#중복 행 지우기
news_df = news_df.drop_duplicates(keep='first',ignore_index=True)
print("중복 제거 후 행 개수: ",len(news_df))

중복 제거 후 행 개수:  0


In [ ]:
print(news_df.head())

Empty DataFrame
Columns: [date, title, link, content]
Index: []


In [ ]:
def make_pd() :
    news_df = pd.DataFrame({'date':news_dates,'title':news_titles,'link':news_urls,'content':news_contents})

    #중복 행 지우기
    news_df = news_df.drop_duplicates(keep='first',ignore_index=True)
    print("중복 제거 후 행 개수: ",len(news_df))

    print(news_df.head())

In [ ]:
t=1
while t<5 :
    # 뉴스의 기본 정보 가져오기
    news_url_a = get_top3_news_info()
    # print(news_info)
    for i in news_url_a:
        # 뉴스 본문 가져오기
        get_news_contents(i)
    
    make_pd()

    t += 1
    time.sleep(10)

AttributeError: 'NoneType' object has no attribute 'find_all'

In [ ]:
# # 함수 호출 - '정치', '경제', '사회' 분야의 상위 3개 뉴스 크롤링
# i=1
# while i<5 :
#     get_naver_news_top3()
#     make_pd()
#     i += 1

ValueError: All arrays must be of the same length